# 제목

# 가. 준비

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import itertools
import numpy as np
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
from pyproj import Transformer
from shapely.geometry import Point, MultiLineString, Polygon, LineString

from geoband.API import *
GetCompasData('SBJ_2102_002', '1', '1.수원시_버스정류장.csv')
GetCompasData('SBJ_2102_002', '2', '2.수원시_버스정류장별_승하차이력(1).csv')
GetCompasData('SBJ_2102_002', '3', '3.수원시_버스정류장별_승하차이력(2).csv')
GetCompasData('SBJ_2102_002', '4', '4.수원시_버스정류장별_승하차이력(3).csv')
GetCompasData('SBJ_2102_002', '5', '5.수원시_버스정류장별_승하차이력(4).csv')
GetCompasData('SBJ_2102_002', '6', '6.수원시_버스정류장별_승하차이력(5).csv')
GetCompasData('SBJ_2102_002', '7', '7.수원시_버스정류장별_노선현황.csv')
GetCompasData('SBJ_2102_002', '14', '14.수원시_시간대별_유동인구(2020).csv')
GetCompasData('SBJ_2102_002', '22', '22.수원시_상세도로망_LV6.geojson')
GetCompasData('SBJ_2102_002', '23', '23.수원시_평일_일별_시간대별_추정교통량_LV6.csv')
GetCompasData('SBJ_2102_002', '24', '24.수원시_평일_일별_혼잡빈도강도_LV6.csv')
GetCompasData('SBJ_2102_002', '25', '25.수원시_평일_일별_혼잡시간강도_LV6.csv')
GetCompasData('SBJ_2102_002', '26', '26.수원시_인도(2017).geojson')

# 나. 버스정류장 골라내기

## 1. 버스정류장

In [ ]:
# 수원시 버스 정류장 데이터 가공
# ===============================
busStop = pd.read_csv('1.수원시_버스정류장.csv')
busStop.info()

In [ ]:
# BIS가 있는 정류장의 이름, 위도, 경도를 추려냄.
busStop = busStop[busStop['BIS설치여부'] == 1][['정류장ID', '정류장명', 'lon', 'lat']]
busStop.index = pd.RangeIndex(len(busStop))

# 정류장의 위도, 경도를 바탕으로 정류장의 위치를 나타내는 Point 객체를 만듦.
# 나중에 버스정류장과 인도 사이의 계산에 필요함.
busStop_point = []
for i in range(len(busStop)):
    busStop_point.append(Point(busStop['lon'][i], busStop['lat'][i]))
busStop['location'] = busStop_point

# 완성된 버스정류장 데이터 확인
busStop

## 2. 인도

In [ ]:
# 수원시 인도 데이터 가공
# ========================
sidewalk = gpd.read_file('26.수원시_인도(2017).geojson')

# 인도 데이터 처리:
#    1. 정류장 세로 폭 2.5m + 보행자 통로 확보 1.5m = 4.0m 이상의 폭(WIDT)를 가진 인도만을 선발함.
#    2. 비포장도로(QUAL SWQ004) 또는 순수 자전거도로(KIND SWK002)는 제외함.
#    3. 최종적으로 선발된 인도의 UFID와 WIDT, geometry 정보만을 저장함.
sidewalk = sidewalk[ (sidewalk['WIDT']>=4) & (sidewalk['QUAL']!='SWQ004') & (sidewalk['KIND']!='SWK002') ][['UFID','WIDT', 'geometry']]
sidewalk.index = pd.RangeIndex(len(sidewalk))

# 가공된 인도 데이터 확인
sidewalk

In [ ]:
# 인도 데이터 추가 처리:
#    제공된 인도의 geometry는 MultiLineString으로 되어있음.
#    이 데이터는 여러 개의 꺾은선 모양으로, 버스정류장의 좌표가 해당 인도에 위치해 있는지 판단하기 매우 어려움.
#    따라서 MultiLineString을 일정 면적을 가진 Polygon으로 변환하여 나중에 쓰기 편하도록 바꿈.
for i in range(len(sidewalk)):
    sidewalk['geometry'][i] = sidewalk['geometry'][i].buffer(0.000135)

## 3. 버스정류장 + 인도

In [ ]:
# 버스정류장 데이터와 인도 데이터 결합
# ====================================

# 버스정류장이 위치한 인도가 4.0m 이상의 폭을 가진 인도인지 확인, 그 결과를 busStop에 추가한다.
hasEnoughSpace = [False for i in range(len(busStop))]

for i in range(len(busStop)):
    for a_sidewalk in sidewalk['geometry']:
        if a_sidewalk.contains(busStop['location'][i]):
            hasEnoughSpace[i] = True

busStop['hasEnoughSpace'] = hasEnoughSpace

# 버스정류장 중 주변공간이 충분치 않은 곳은 제외한다.
busStop = busStop[busStop['hasEnoughSpace']==True]
busStop = busStop.iloc[:, 0:4]
busStop.index = pd.RangeIndex(len(busStop))

# 완성된 버스정류장 데이터 확인
#busStop.to_csv('버스정류장_BIS있음_인도폭4m이상.csv')
busStop

## 4. 시각화

In [ ]:
# 시각화(필수아님)
# ======================

m = folium.Map(
    location = [37.27704264490948, 127.00430215626447], #수원여고 좌표.
    zoom_start = 13
)

folium.Choropleth(
    geo_data = sidewalk,
    data = sidewalk[['UFID', 'WIDT']],
    columns = ('UFID', 'WIDT'),
    key_on = 'feature.properties.UFID',
    fill_color = 'Pastel2',
    fill_opacity = 0.7,
    line_opacity = 1.0,
    legend_name = '인도 폭 (m)'
).add_to(m)

for i in range(len(busStop)):
    folium.Marker(
        location = [busStop['lat'][i], busStop['lon'][i]],
        popup = busStop['정류장명'][i],
        icon = folium.Icon(icon = 'bus', prefix = 'fa')
    ).add_to(m)

m

# 다. 승차이력 점수 부여하기

## 1. 정류장별 전체 승차건수 평균

In [ ]:
df2 = pd.read_csv("2.수원시_버스정류장별_승하차이력(1).csv")
df3 = pd.read_csv("3.수원시_버스정류장별_승하차이력(2).csv")
df4 = pd.read_csv("4.수원시_버스정류장별_승하차이력(3).csv")
df5 = pd.read_csv("5.수원시_버스정류장별_승하차이력(4).csv")
df6 = pd.read_csv("6.수원시_버스정류장별_승하차이력(5).csv")

# 모든 승하차 이력 합치기
df = pd.concat([df2,df3,df4,df5,df6])

In [ ]:
# 정류소ID 별로 전체 승하차 건수 확인
df_riding = df.groupby(by=['정류소ID'], as_index=False).sum()
df_riding = df_riding[['정류소ID', '전체 승차 건수']]
df_riding.columns.values[0] = '정류장ID'
df_riding

In [ ]:
# 전체 승차 건수 이상치 확인 : X/MAX 방식으로 가중치 산정 근거
plt.boxplot(df_riding['전체 승차 건수'],
           notch=1, 
            sym='bo', 
            vert=0 
           )

## 2. 승차건수별 점수 계산(비율)

In [ ]:
# 나. 에서 구한 busStop의 500개 정류장의 데이터만 가져옴.
temp = pd.merge(busStop['정류장ID'], df_riding, on='정류장ID', how='left')

In [ ]:
# 점수 계산
temp['점수'] = round(temp['전체 승차 건수']/max(temp['전체 승차 건수']), 4)
temp

In [ ]:
# busStop에 해당 점수 반영
busStop = busStop.assign(승차건수_점수 = temp['점수'])
busStop

# 라. 노선에 따른 점수 부여하기

## 1. 정류장별 운행노선의 개수 계산

In [ ]:
df7 = pd.read_csv('7.수원시_버스정류장별_노선현황.csv', encoding = 'UTF-8')

cnt = []

for i in range(len(df7)):
    cnt.append(df7['운행노선'][i].count(',') +1)

df7 = df7[['정류소명']]
df7 = df7.assign(운행노선_수 = cnt)
df7.columns.values[0] = '정류장명'
df7

In [ ]:
# 이름이 같은 정류장들은 평균치를 냈음(자료에 정확한 위치가 주어져 있지 않아 판단 힘듦)
df7 = df7.groupby(['정류장명']).mean().reset_index()

In [ ]:
df7

In [ ]:
df7['운행노선_수'].describe()

In [ ]:
# 운행 노선_수 이상치 확인 : X/MAX 방식으로 가중치 산정 근거
plt.boxplot(df7['운행노선_수'],
            notch=1, 
            sym='bo', 
            vert=0 
           )

## 2. 점수 계산

In [ ]:
# 나., 다. 에서 구한 busStop에 해당하는 정류장만 가져옴
temp = pd.merge(busStop['정류장명'], df7, on = '정류장명', how = 'left')

In [ ]:
# 점수를 구해서 busStop에 추가함
score = round(temp['운행노선_수']/max(temp['운행노선_수']), 4)

busStop = busStop.assign(운행노선_점수 = score)
busStop

# 마. 유동인구 수 점수 부여하기

## 1. 버스 운행 시간대(0500-2259)의 평균 유동인구 구하기 

In [ ]:
# 유동인구 데이터 처리
# ====================
df14 = pd.read_csv('14.수원시_시간대별_유동인구(2020).csv')
temp = pd.DataFrame(df14[['lon', 'lat']])
temp['유동인구'] = pd.DataFrame(df14.iloc[:, 6:24].mean(axis=1))

floating_population = temp.groupby(['lon', 'lat']).mean().reset_index()
floating_population

In [ ]:
# 유동 인구 수 이상치 확인 : X/MAX 방식으로 가중치 산정 근거
plt.boxplot(floating_population['유동인구'],
            notch=1, 
            sym='bo', 
            vert=0 
           )

In [ ]:
# 시간대를 버스가 다니는 시간대인 05시부터 23시 이전까지로 정한다.
# 해당 시간대의 유동인구 평균을 구해 특정 날짜, 특정 위치의 유동인구값을 구한다.
# 위치를 바탕으로 묶어서 위치별 평균 유동인구 값을 구한다.


points=[]
for i in range(len(floating_population)):
    points.append(Point(floating_population.loc[i,'lon'], floating_population.loc[i,'lat']))

floating_population = floating_population.assign(location=points)

# 완성된 유동인구 데이터 확인
floating_population

## 2. 유동인구 데이터 시각화

In [ ]:
# 유동인구 데이터 시각화
# ======================

# 수원시를 1km x 1km 격자로 나눈다. (나눈 값들은 polygons에 저장된다)
#    folium에서는 epsg:4326 을 쓰고, 미터 좌표계는 epsg:5186 이다.
#    따라서 좌표계를 변환해주는 장치가 필요하다.

folium_to_meter = Transformer.from_crs('epsg:4326', 'epsg:5186', always_xy=True)
meter_to_folium = Transformer.from_crs('epsg:5186', 'epsg:4326', always_xy=True)
base = (126.92877511736562, 37.228786072536124) # 대충 수원시 서남단 구석
unit = 1000 # 1000m 라는 뜻 ## << 500m 로 하고싶으면: 500
polygons = []
x, y = folium_to_meter.transform(base[0], base[1]) # base좌표를 meter좌표계로 바꿈.

for i, j in itertools.product(range(15), range(12)): ## << 500m 로 하고싶으면: 30, 24
    point1 = meter_to_folium.transform(x + (i+0)*unit, y + (j+0)*unit)
    point2 = meter_to_folium.transform(x + (i+1)*unit, y + (j+0)*unit)
    point3 = meter_to_folium.transform(x + (i+1)*unit, y + (j+1)*unit)
    point4 = meter_to_folium.transform(x + (i+0)*unit, y + (j+1)*unit)
    polygons.append(Polygon([point1, point2, point3, point4]))

#    각 grid들을 저장한다.
idxes = [i for i in range(len(polygons))]
temp = {'idx':idxes, 'geometry':polygons, 'count':0, 'fp_sum':0, 'floatingPopulation':0}
suwon_grid = pd.DataFrame(temp)

In [ ]:
# 유동인구 데이터는 point형태로 값이 주어져 있으므로, 그 point가 어느 격자에 속하는지를 판단한다.
for i in range(len(floating_population)):
    for j in range(len(suwon_grid)):
        if suwon_grid.loc[j,'geometry'].contains(floating_population.loc[i,'location']):
            suwon_grid.loc[j, 'count'] += 1
            suwon_grid.loc[j, 'fp_sum'] += floating_population.loc[i, '유동인구']
            break

In [ ]:
# 각 grid마다의 평균 유동인구를 구한다.
suwon_grid['floatingPopulation'] = suwon_grid['fp_sum'] / suwon_grid['count']

In [ ]:
# 시각화를 위해 GeoDataFrame 객채를 만든다.
geo_suwon_grid = gpd.GeoDataFrame(suwon_grid, geometry = suwon_grid['geometry'])
geo_suwon_grid.set_crs(epsg=4326, inplace=True)
print('done')

In [ ]:
# 시각화
m = folium.Map(location = [37.27704264490948, 127.00430215626447], zoom_start=12)

folium.Choropleth(
    geo_data = geo_suwon_grid,
    data = suwon_grid,
    columns = ('idx', 'floatingPopulation'),
    key_on = 'feature.properties.idx',
    fill_opacity = 0.8,
    line_opacity = 0.2
).add_to(m)

m

## 3. 버스정류장 별 유동인구 점수 계산

In [ ]:
temp = busStop[['정류장ID', 'lon', 'lat']]

# 정류장이 격자의 어느 부분에 속해있는지를 판단, 해당 격자의 유동인구 값을 정류장에게 부여한다.
fp=[]
for busstop in range(len(temp)):
    for grid in range(len(suwon_grid)):
        if Point( temp.loc[busstop,'lon'], temp.loc[busstop,'lat'] ).within(suwon_grid.loc[grid,'geometry']):
            fp.append(suwon_grid.loc[grid, 'floatingPopulation'])

In [ ]:
# 점수를 구한다.
fp_score = np.round(fp / max(fp), 4)
busStop = busStop.assign(유동인구_점수 = fp_score)
busStop

# 바. 추정교통량 점수 부여하기

## 1. 데이터 처리

In [ ]:
#df22 = gpd.read_file('22.수원시_상세도로망_LV6.geojson')
df23 = pd.read_csv(('23.수원시_평일_일별_시간대별_추정교통량_LV6.csv'),
                  dtype={"시간적범위": "string"})
df24 = pd.read_csv('24.수원시_평일_일별_혼잡빈도강도_LV6.csv')
df25 = pd.read_csv('25.수원시_평일_일별_혼잡시간강도_LV6.csv')

In [ ]:
# 22번데이터
상세도로망 = gpd.read_file('22.수원시_상세도로망_LV6.geojson')
상세도로망.info()
상세도로망

In [ ]:
# 23번 데이터의 종류 확인
df23['시간적범위'].value_counts()

In [ ]:
# 시간범위를 fulltime만 사용
df23_교통량 = df23[df23['시간적범위'] == 'fulltime']
df23_교통량

In [ ]:
#24번, 25번 데이터의 정보
df24.info()
df25.info()

In [ ]:
# 24번, 25번 데이터에서 사용할 데이터 추려내기
혼잡빈도강도_전체 = df24[['상세도로망_LinkID','혼잡빈도강도']]
혼잡시간강도_전체 = df25[['상세도로망_LinkID','혼잡시간강도']]

In [ ]:
# 23번 데이터에 추려낸 혼잡시간,빈도 강도 합치기.
df23_교통량 = df23_교통량.merge(혼잡빈도강도_전체,on='상세도로망_LinkID',how="outer").merge(혼잡시간강도_전체,on='상세도로망_LinkID',how="outer")
df23_교통량['혼잡시간강도'].fillna(value = df23_교통량.median()['혼잡시간강도'], inplace=True)
df23_교통량['혼잡빈도강도'].fillna(value = df23_교통량.median()['혼잡빈도강도'], inplace=True)
df23_교통량 = df23_교통량.drop(['승용차_추정교통량','버스_추정교통량','화물차_추정교통량'],axis=1)
df23_교통량

In [ ]:
# df23 상데도로망과 읍면동명으로 groupby()
df23_교통량 = df23_교통량.groupby(['상세도로망_LinkID','읍면동명'], as_index=False).mean()
df23_교통량 = df23_교통량.sort_values(by='전체_추정교통량' ,ascending=False)
df23_교통량

In [ ]:
# 전체_추정교통량 확인 : X/MAX 방식으로 가중치 산정 근거
plt.boxplot(df23_교통량['전체_추정교통량'],
            notch=1, 
            sym='bo', 
            vert=0 
           )

In [ ]:
# 혼잡시간강도 : Natural break 방식으로 가중치 산정 근거
plt.boxplot(df23_교통량['혼잡시간강도'],
            notch=1, 
            sym='bo', 
            vert=0 
           )

In [ ]:
# 혼잡빈도강도: Natural break 방식으로 가중치 산정 근거
plt.boxplot(df23_교통량['혼잡빈도강도'],
            notch=1, 
            sym='bo', 
            vert=0 
           )

In [ ]:
교통량_max=max(df23_교통량['전체_추정교통량'])
시간강도_max=max(df23_교통량['혼잡빈도강도'])
빈도강도_max=max(df23_교통량['혼잡시간강도'])
print(교통량_max)
print(시간강도_max)
print(빈도강도_max)
df23_교통량

In [ ]:
df23_교통량['가중치']=df23_교통량['전체_추정교통량']/교통량_max
빈도가중치 = jenkspy.jenks_breaks(df23_교통량['혼잡빈도강도'], nb_class =5)
시간가중치 = jenkspy.jenks_breaks(df23_교통량['혼잡시간강도'], nb_class =5)
df23_교통량['빈도가중치']=pd.cut(df23_교통량['혼잡빈도강도'],bins=빈도가중치, labels=[0.2,0.4,0.6,0.8,1.0],
                        include_lowest=True)
df23_교통량['시간가중치']=pd.cut(df23_교통량['혼잡시간강도'],bins=시간가중치, labels=[0.2,0.4,0.6,0.8,1.0],
                        include_lowest=True)
df23_교통량 = df23_교통량.astype(
    {
        '빈도가중치':np.float,
        '시간가중치':np.float,
        '가중치':np.float
    }
)
df23_교통량

In [ ]:
df23_교통량['가중치']=df23_교통량['가중치']+df23_교통량['빈도가중치']+df23_교통량['시간가중치']
가중치최대=max(df23_교통량['가중치'])

print(가중치최대)
df23_교통량['가중치']=round(df23_교통량['가중치']/가중치최대,2)
print(max(df23_교통량['가중치']))

In [ ]:
df23_교통량

## 3. 시각화

In [ ]:
# df23번 LinkID 상행, 하행 데이터로 구분
df23_교통량['상세도로망_LinkID'] = df23_교통량['상세도로망_LinkID'].apply(str)
df23_교통량['up_down'] = df23_교통량['상세도로망_LinkID'][:].str[9:11].replace({'01': '상행','02':'하행'})
df23_교통량['link_id'] = df23_교통량['상세도로망_LinkID'][:].str[0:9]
df23_교통량.info()
df23_교통량

In [ ]:
#]link_id, 읍면동명]groupby / 전체 추정 교통량 내림차순으로 정렬
df23_교통량최종 = df23_교통량.groupby(['link_id','읍면동명'], as_index=False).mean()
df23_교통량최종 = df23_교통량.sort_values(by='전체_추정교통량', ascending=False)
df23_교통량최종 = pd.DataFrame(df23_교통량최종)
df23_교통량최종.info()
# 사용한 가중치 제거 * 가중치 = (시간강도+혼잡빈도+교통량가중치)/max((시간강도+혼잡빈도+교통량가중치))
df23_교통량최종 = df23_교통량최종.drop(['빈도가중치','시간가중치'],axis=1)
df23_교통량최종

In [ ]:
# 도로에 buffer를 씌워서 MultiPolygon 생성
for i in range(len(상세도로망)):
     상세도로망['geometry'][i] = 상세도로망['geometry'][i].buffer(0.00015)

In [ ]:
# 혼잡한 도로 시각화
m = folium.Map(location = [37.26369641371368, 127.02856264231166], zoom_start=11)

folium.Choropleth(
    geo_data = 상세도로망,
    data = df23_교통량최종, 
    columns = ['link_id', '가중치'],
    key_on = 'feature.properties.link_id',
    fill_color = 'Blues',
    fill_opacity = 0.5,
    line_opacity = 0.5).add_to(m)

m

## 4. 버스정류장에 점수 부여하기

In [ ]:
# 가중치를 구하기 위해 버퍼값을 늘림.
상세도로망_점수 = 상세도로망
상세도로망_점수
for i in range(len(상세도로망_점수)):
     상세도로망_점수['geometry'][i] = 상세도로망_점수['geometry'][i].buffer(0.00035)

In [ ]:
#link_id 기준으로 df23_교통량 최종, 상세도로망 merge
도로구역 = df23_교통량최종.merge(상세도로망_점수, on='link_id',how='left') 
도로구역.drop('상세도로망_LinkID', axis=1,inplace=True)
도로구역 = 도로구역[['link_id','읍면동명','도로등급','전체_추정교통량','혼잡빈도강도','혼잡시간강도','가중치','geometry']]
도로구역

In [ ]:
temp = busStop[['정류장ID', 'lon', 'lat']]

In [ ]:
score = []

# floating_population.loc[i,'location']은 point형태로 되어있음.
# 유동인구 데이터는 point형태로 값이 주어져 있으므로, 그 point가 어느 격자에 속하는지를 판단한다.
for i in range(len(busStop)):
    # 해당 격자내에 point가 있으면 모든격자 탐색하지 않고 break
    for j in range(len(도로구역)):
        if 도로구역.loc[j,'geometry'].contains(Point(temp['lon'][i], temp['lat'][i])):
            score.append(도로구역.loc[j,'가중치'])
            break

In [ ]:
busStop = busStop.assign(교통량_점수 = score)
busStop

In [ ]:
def calculate(score1, score2, score3, score4):
    result = score1*0.3 + score2*0.2 + score3*0.1+ score4*0.4
    return result

In [ ]:
final_score = [0 for i in range(len(busStop))]
final_score = calculate(busStop['승차건수_점수'], busStop['운행노선_점수'], busStop['유동인구_점수'], busStop['교통량_점수'])
final_score = final_score*100

In [ ]:
final = busStop.assign(최종점수 = final_score)

In [ ]:
final = final.sort_values(by='최종점수' ,ascending=False)

In [ ]:
df = final.head(30)

In [ ]:
df = df.reset_index(drop=True)
import import_ipynb
from AdTarget import AdTarget

In [ ]:
adTarget = AdTarget()
adTarget.arrangeData()

In [ ]:
target = []
for i in range(len(df)):
    target.append(adTarget.getAdTarget(df['lon'][i], df['lat'][i]))
df = df.assign(광고_타겟층 = target)
df2 = df[['정류장ID', '정류장명', 'lon', 'lat', '광고_타겟층']]
df2